In [1]:
import pandas as pd #handle files, database etc.

In [3]:
dataset = pd.read_csv("Social_Network_Ads.csv") #read the csv file using pandas library

dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [5]:
#data preprocessing - because categorial data can't be handle so we modify the data into numerical data using one hot encoding(Nominal)

dataset = pd.get_dummies(dataset, drop_first=True) #"drop_first=True" removes 1st category of each categorical column

dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [7]:
boolean_columns = dataset.select_dtypes(include=['bool']).columns #get boolean type column data from the dataset and store it in boolean_columns variable
#print(boolean_columns)

dataset[boolean_columns] = dataset[boolean_columns].astype(int) #convert boolean column dataset to 0's and 1's
#print(dataset[boolean_columns])

In [9]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [11]:
dataset = dataset.drop("User ID", axis=1) #removing "User ID" feature using drop method. "axis=1" represent removing the "COLUMN"

dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [13]:
dataset["Purchased"].value_counts() #value_counts method calculates the purchased and not purchased value of "Purchased" column

Purchased
0    257
1    143
Name: count, dtype: int64

In [15]:
dataset.columns #return column names present in the dataset variable

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [17]:
independent = dataset[['Age', 'EstimatedSalary', 'Gender_Male']] #input features

dependent = dataset[['Purchased']] #output feature

In [19]:
#split training and testing dataset

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(independent, dependent, test_size=0.30, random_state=0)

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

parameters = {'n_estimators':[10,50,100],
             'criterion':['gini','entropy','log_loss'],
             'max_features':['sqrt','log2'],
             'min_samples_split': [3]}
             #'ccp_alpha':[0.1, 0.2]}

grid = GridSearchCV(RandomForestClassifier(), parameters, refit=True, n_jobs=-1, scoring='f1', verbose=3)
grid.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Users\sethu\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_split': [3],
                         'n_estimators': [10, 50, 100]},
             scoring='f1', verbose=3)

In [23]:
classifier = grid.cv_results_

#print("CV_RESULTS : ", re)

grid_pred = grid.predict(X_test) #test the model with test data

from sklearn.metrics import confusion_matrix #Evaluation metrics

cm = confusion_matrix(y_test, grid_pred)
print("Confusion Matrix : ", cm) 

print("Best Score:", grid.best_score_)
print("Best Parameters:", grid.best_params_)

Confusion Matrix :  [[73  6]
 [ 5 36]]
Best Score: 0.859480032162959
Best Parameters: {'criterion': 'entropy', 'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 50}


In [25]:
table = pd.DataFrame.from_dict(classifier) #convert dictonary format to DataFrame(table)

table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.054744,0.008622,0.020204,0.003353,gini,sqrt,3,10,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.789474,0.780488,0.790698,0.909091,0.904762,0.834902,0.058929,15
1,0.183354,0.007698,0.025991,0.002626,gini,sqrt,3,50,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.820513,0.800000,0.761905,0.883721,0.952381,0.843704,0.067158,11
2,0.336094,0.012427,0.026269,0.003723,gini,sqrt,3,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.820513,0.800000,0.790698,0.909091,0.930233,0.850107,0.057992,7
3,0.044023,0.004925,0.016049,0.002345,gini,log2,3,10,"{'criterion': 'gini', 'max_features': 'log2', ...",0.789474,0.857143,0.761905,0.909091,0.926829,0.848888,0.064596,9
4,0.180257,0.014439,0.021565,0.000797,gini,log2,3,50,"{'criterion': 'gini', 'max_features': 'log2', ...",0.820513,0.800000,0.790698,0.909091,0.952381,0.854536,0.064431,5
5,0.334273,0.018398,0.029591,0.002221,gini,log2,3,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.820513,0.780488,0.790698,0.909091,0.930233,0.846204,0.061766,10
6,0.041033,0.010023,0.017138,0.004958,entropy,sqrt,3,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.789474,0.769231,0.761905,0.954545,0.904762,0.835983,0.078606,14
7,0.182136,0.012143,0.023068,0.003768,entropy,sqrt,3,50,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.820513,0.829268,0.761905,0.933333,0.952381,0.859480,0.072162,1
8,0.337248,0.008969,0.026472,0.003416,entropy,sqrt,3,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.820513,0.829268,0.790698,0.909091,0.930233,0.855960,0.053977,4
9,0.045080,0.003122,0.018978,0.001866,entropy,log2,3,10,"{'criterion': 'entropy', 'max_features': 'log2...",0.756757,0.700000,0.790698,0.857143,0.930233,0.806966,0.079925,17


In [27]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_pred) #generation classification report

print(clf_report)

              precision    recall  f1-score   support

           0       0.94      0.92      0.93        79
           1       0.86      0.88      0.87        41

    accuracy                           0.91       120
   macro avg       0.90      0.90      0.90       120
weighted avg       0.91      0.91      0.91       120



In [29]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [31]:
#save the model using pickle library

import pickle
fileName = "finalized_model_random_forest_regression.sav"
pickle.dump(grid, open(fileName, "wb")) #saved model available in regressor variable. Now storing that variable into fileName

In [33]:
loaded_model = pickle.load(open("finalized_model_random_forest_regression.sav", "rb"))
result = loaded_model.predict([[30, 70000, 0]])

result

C:\Users\sethu\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([0], dtype=int64)